# Bitinfo for ERA5-Land data
Here we determine the keepbits for 99% real information using xbitinfo

In [ ]:
import xarray as xr
import xbitinfo as xb
import hvplot.xarray
import fsspec 

In [ ]:
nc3_file = 'era5land_1980-12-01.nc'
nc4_file = 'era5land_1980-12-01-c99.nc'

In [ ]:
ds = xr.open_dataset(nc3_file, chunks={})

In [ ]:
ds

In [ ]:
bitinfo = xb.get_bitinformation(ds, dim="time")  
keepbits = xb.get_keepbits(bitinfo, inflevel=0.99)  # get number of mantissa bits to keep for 99% real information

In [ ]:
print(keepbits)

In [ ]:
keepbits.to_netcdf('keepbits.nc', mode='w')

In [ ]:
ds_bitrounded = xb.xr_bitround(ds, keepbits)
encoding = {}
for data_var in ds.data_vars:
    encoding[data_var]=dict(dtype='float32', zlib=True)

encoding['latitude'] = {'_FillValue':None}
encoding['longitude'] = {'_FillValue':None}

ds_bitrounded.to_netcdf(nc4_file, engine='netcdf4', encoding=encoding, mode='w')  

In [ ]:
!du -hs *.nc